# CNN to detect and localize annotations in a given image file.

In [3]:
# Code to check the resources available Ex. GPU RAM
'''
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()
'''


Gen RAM Free: 12.7 GB  | Proc size: 138.7 MB
GPU RAM Free: 11438MB | Used: 1MB | Util   0% | Total 11439MB


In [2]:
# Installing PyDrive
!pip install PyDrive

    100% |████████████████████████████████| 993kB 7.4MB/s 
  Running setup.py bdist_wheel for PyDrive ... - \ done
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Insert zip folder with all training images in it. Then unzip it
fileId = drive.CreateFile({'id': '19kmV8qjzVzUj02TwHS1Uw7BzdB63dF_-'}) # id is the id of the file being uploaded
print (fileId['title'])  # dataset_train.zip
file = fileId['title']
fileId.GetContentFile(file)  # Save Drive file as a local file


dataset_train.zip


In [0]:
# Unzip the datafile
!unzip dataset_train.zip -d ./

In [1]:
!ls
!rm dataset_train.zip
!ls

adc.json  datalab  dataset_test  dataset_train	sample_data
rm: cannot remove 'dataset_train.zip': No such file or directory
adc.json  datalab  dataset_test  dataset_train	sample_data


# Installing PyTorch & OpenCV

In [7]:
# Install pytorch
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x5bf3c000 @  0x7f847e34e1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8


In [2]:
#Check install and version
import torch
torch.__version__

# Check cuda avaialability
if torch.cuda.is_available():
   print("Yay!!")
    

Yay!!


In [9]:
# Install openCV
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

# Installing Pillow as py torch transform dependencies
!pip install Pillow==4.1.1
!pip install image

    100% |████████████████████████████████| 5.7MB 4.5MB/s 
  Found existing installation: Pillow 5.2.0
    Uninstalling Pillow-5.2.0:
      Successfully uninstalled Pillow-5.2.0
    100% |████████████████████████████████| 7.3MB 4.0MB/s 


In [0]:
import os
import cv2

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from PIL import Image
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [0]:
# Custom Dataset created to load training and testing data

class ImgDataset(Dataset):
    def __init__(self, data_dir, transform = None, 
                 img_shape = (600,600), grid_dim = (30,30)):
        
        # Length of per grid vector, based on number of classes being detected
        self.vector_len = 5 
        
        self.img_shape = img_shape
        self.grid_dim = grid_dim
        self.transform = transform
        
        
        # Create a grid. Grid doesnt change as image size is same
        self.grid_list, self.col_width, self.row_height = \
        self.create_grid(self.img_shape, self.grid_dim)
        
        # Fetch all files from dataset directory
        self.files = os.listdir(data_dir)
        self.files = [os.path.join(data_dir,f) for f in self.files]

        # Make separate list for images and text files
        self.img_list = [f for f in self.files if '.jpg' in f]
        self.txt_list = [f for f in self.files if '.txt' in f]
        self.labels = []
        
        for txt_file in self.txt_list:
            # Call function to iterate over grid_list 
            # and generate label vector for the image
            img_label = self.label_vector_gen(txt_file)
            # Has one-to-one correspondence with img_list
            self.labels.append(img_label) 
        

    def __len__(self):
        '''Return size of the dataset'''
        return len(self.img_list)
    
    def __getitem__(self,idx):
        '''Enable indexing on dataset'''
        # Processing the image
        img = Image.open(self.img_list[idx]).convert('RGB')  # PIL image
        image = self.transform(img)
        
        # Processing the label for that corresponding image
        label = self.labels[idx]
        label = torch.FloatTensor(label)
        # Flatenning the label into [4500] size vector
        label = label.view(-1)
        return image, label

    def create_grid(self,img_shape,grid_dim):
        '''
        Function: 
            Function creates a grid with cells of equal size.
        
        Parameters:
            img_shape : dimensions of the image (height,width).
            grid_dim  : dimensions of the grid (rows,colms).
            Note: width must be divisible by no. of columns
            and height must be divisible by no. of rows
            
        Returns:
            This function returns a list of top left coordinates of cells, 
            along with the col_width and row_height.
        '''
        height, width = img_shape
        rows,colms = grid_dim
        
        col_width = width//colms
        
        row_height = height//rows
        
        grid_list = []
        
        for c in range(0,width,col_width): # Step size is the col_width
            for r in range(0,height,row_height): # Step size is the row_height
                grid_list.append((c,r))
                
        return grid_list, col_width, row_height


    def label_vector_gen(self,txt_file):
        '''
        Function: 
            Function generates the label vector for each image.
        
        Parameters:
            txt_file  : Name of file name that has the bounding box data.
        Returns:
            This function returns the label vector for each image.
            Note: Each label is of size
            grid_rows x grid_colmn x no.of classes x len of vector
            = len(grid_list) x 1 x 5
        '''
        
        # Open file to read box details
        with open(txt_file, 'r') as file:
            bounding_boxes = file.read().split('\n')
            # Remove last emelent as it is empty
            bounding_boxes = bounding_boxes[:-1]
        
        coordinate_list = [] # Store coordinates of cells with bbox center 
        label = [] # Store the labels for each bounding box
        
        if len(bounding_boxes)>0:
            for box in bounding_boxes:
                box = box.split(' ')
                # Each element of box list has one attribute of the box as str
                # un-normalize them by multiplying width and height
                box_center_x = float(box[1]) * self.img_shape[1]
                box_center_y = float(box[2]) * self.img_shape[0]
                
                # Get the coordinate of the cell containing the box centers
                grid_x = (box_center_x//self.col_width) * self.col_width
                grid_y = (box_center_y//self.row_height) * self.row_height
                coordinate_list.append((grid_x,grid_y))
                
                
            # Check if point lies in cell and generate the vector
            for cell in self.grid_list:
                if cell in coordinate_list:
                    # Make vector as required
                    idx = coordinate_list.index(cell)
                    box_info = bounding_boxes[idx] # Get corresponding box data
                    box_info = box_info.split(' ')
                    
                    class_pred = float(box_info[0]) 
                    x_center = float(box_info[1])
                    y_center = float(box_info[2])
                    width = float(box_info[3])
                    height = float(box_info[4])
                    
                    label.append((class_pred, 
                                  x_center, 
                                  y_center, 
                                  width, 
                                  height))
                else:
                    # 99 means no value as object doesnt exist in the grid
                    # (class_pred, x_center, y_center, width, height)
                    label.append((99,99,99,99,99))
                    
        elif len(bounding_boxes) == 0:
            label = [(99,99,99,99,99)] * (self.grid_dim[0] * self.grid_dim[1])
            
        return label

In [0]:
# Loading trainig data

# Normalizing the images
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])

# creating transform, resizing again (can be removed)
transform = transforms.Compose([transforms.Resize((600,600)),
                                transforms.ToTensor(),
                                normalize])

# Creating dataset object
train_dataset = ImgDataset('dataset_train', transform = transform, 
                 img_shape = (600,600), grid_dim = (30,30))

# Creating dataloader to load a batch of 4 images at once
train_dataloader = DataLoader(train_dataset,
                              batch_size=8, 
                              shuffle=True,
                              num_workers=0,
                              pin_memory=True)

In [0]:
# Create Convolutional Neural Net to detect and localize annotations
# Net should take an image as input and give out a vector equivalent 
# to the label vector as output.
# Input image shape (h=600,w=600,c=3) in batch: 4x3x600x300

# Net should give 900x5 output as of now
# As a 60x30 grid with 5 features per cell = 900x5
# Input batch will have size batch_size x 900x5

# Using YOLO like architecture
# 9 convolutoion layers and one fully connected layer
# Kernel for convolution = 3x3, stride = 1
# Kernel for MaxPooling = 2x2, stride = 2

class CNN(nn.Module):
  
  def __init__(self):
    super(CNN,self).__init__()
    
    # Convolution layer 1
    # Out channels is the number of filters
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=16,
                           kernel_size=3, padding =1)
    self.bn1 = nn.BatchNorm2d(16)
    
    # Convolution layer 2
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=32,
                           kernel_size=3, padding =1)
    self.bn2 = nn.BatchNorm2d(32)
    
    # Convolution layer 3
    self.conv3 = nn.Conv2d(in_channels=32, out_channels=64,
                           kernel_size=3, padding =1)
    self.bn3 = nn.BatchNorm2d(64)
    
    # Convolution layer 4
    self.conv4 = nn.Conv2d(in_channels=64, out_channels=128,
                           kernel_size=3, padding =1)
    self.bn4 = nn.BatchNorm2d(128)
    
    # Convolution layer 5
    self.conv5 = nn.Conv2d(in_channels=128, out_channels=256,
                           kernel_size=3, padding =1)
    self.bn5 = nn.BatchNorm2d(256)
    
    # Convolution layer 6
    self.conv6 = nn.Conv2d(in_channels=256, out_channels=512,
                           kernel_size=3, padding =1)
    self.bn6 = nn.BatchNorm2d(512)
    
    # Convolution layer 7
    self.conv7 = nn.Conv2d(in_channels=512, out_channels=1024,
                           kernel_size=3, padding =1)
    self.bn7 = nn.BatchNorm2d(1024)
    
    # Convolution layer 8
    self.conv8 = nn.Conv2d(in_channels=1024, out_channels=1024,
                           kernel_size=3, padding =1)
    self.bn8 = nn.BatchNorm2d(1024)
    
    # Convolution layer 9
    self.conv9 = nn.Conv2d(in_channels=1024, out_channels=1024,
                           kernel_size=3, padding =1)
        
    
    # Fully connected Layer
    self.fc1 = nn.Linear(in_features=82944, out_features=30*30*5)
    
    # Now we get 
    
  def forward(self,x):
    x= self.bn1(self.conv1(x))
    x= F.max_pool2d(x,kernel_size=2,stride=2)
    x= F.leaky_relu(x,inplace=False)
    
    x= self.bn2(self.conv2(x))
    x= F.max_pool2d(x,kernel_size=2,stride=2)
    x= F.leaky_relu(x,inplace=False)
    
    x= self.bn3(self.conv3(x))
    x= F.max_pool2d(x,kernel_size=2,stride=2)
    x= F.leaky_relu(x,inplace=False)
    
    x= self.bn4(self.conv4(x))
    x= F.max_pool2d(x,kernel_size=2,stride=2)
    x= F.leaky_relu(x,inplace=False)
    
    x= self.bn5(self.conv5(x))
    x= F.max_pool2d(x,kernel_size=2,stride=2)
    x= F.leaky_relu(x,inplace=False)
    
    x= self.bn6(self.conv6(x))
    x= F.max_pool2d(x,kernel_size=2,stride=2)
    x= F.leaky_relu(x,inplace=False)
    
    
    x= self.bn7(self.conv7(x))
    x= self.bn8(self.conv8(x))
    x= self.conv9(x)
    
    # Flattening the output before it hits fully connected layer
    x = x.view(x.size(0), -1) 
    #x= x.view(-1) 
    x= self.fc1(x)
    return x

In [7]:
# Setting the device that will be used for training.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Creating the model instance. 
model = CNN().to(device)

cuda:0


In [0]:
# Loss function and optimizer

criterion = nn.L1Loss() # Loss Function

# We are using Stochastic Gradient descent as our optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [0]:
# Training loop

def train(epochs):
  model.train()
  
  for epoch in range(epochs):
    
    
    # As dataloader has batch_size = 4
    # We will get one batch of 4 images with each iteration 
    for i, (images_batch, target) in enumerate(train_dataloader):

      # Convert images_batch and target to pytorch Variable
      images_batch, target_batch = Variable(images_batch),Variable(target)

      optimizer.zero_grad() # Make sure gradients are initially 0
      
      # Converting input to type torch.cuda
      images_batch.requires_grad_()
      images_batch = images_batch.to(device)
      
      # Converting targets to type torch.cuda
      #target.requires_grad_()
      target = target.to(device)
      
      out = model(images_batch) # Forward pass

      loss = criterion(out, target) # Computing the loss
      loss.backward() # Back-Prop the loss / Backward Pass

      optimizer.step() # Update the gradients
      
      print(f'Batch : {i+1} Loss : {loss}')

    print(f'Epoch : {epoch+1} Loss : {loss}')

In [0]:
train(200)

In [15]:
# Insert zip folder with all validation images in it. Then unzip it
fileId = drive.CreateFile({'id': '1SF7FfUuuB5ragXpFZf4a0KP8xZsCoc6w'}) # id is the id of the file being uploaded
print (fileId['title'])  # dataset_test.zip
file = fileId['title']
fileId.GetContentFile(file)  # Save Drive file as a local file


dataset_test.zip


In [16]:
# Unzip the datafile
!unzip dataset_test.zip -d ./

Archive:  dataset_test.zip
   creating: ./dataset_test/


In [17]:
!ls
!rm dataset_test.zip
!ls

adc.json  datalab  dataset_test  dataset_test.zip  dataset_train  sample_data
adc.json  datalab  dataset_test  dataset_train	sample_data


In [0]:
# Loading the validation set

# Normalizing the images
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])

# creating transform, resizing again (can be removed)
transform = transforms.Compose([transforms.Resize((600,600)),
                                transforms.ToTensor(),
                                normalize])

# Creating dataset object
val_dataset = ImgDataset('dataset_test', transform = transform, 
                 img_shape = (600,600), grid_dim = (30,30))

# Creating dataloader to load a batch of 1 images at once, as the number of images in val set keep changing
# Fix it to 4 or 8 once the val set is fixed.
val_dataloader = DataLoader(train_dataset,
                              batch_size=1, 
                              shuffle=True,
                              num_workers=0,
                              pin_memory=True)

In [0]:
# Validating the model

def validate(epochs):
  model.eval()
  loss = 0
  correct = 0
  
  for epoch in range(epochs):
    
    
    # As dataloader has batch_size = 4
    # We will get one batch of 4 images with each iteration 
    for i, (images_batch, target) in enumerate(val_dataloader):

      # Convert images_batch and target to pytorch Variable
      images_batch, target = Variable(images_batch),Variable(target)

      # setting this context handler ensures that the loss is not
      # backpropagated and we don't run out of GPU memory.
      # Note: if we still run out of GPU memory then decrease batch size
      # in the dataLoader (val_loder in this case)
      
      with  torch.no_grad():
        # Converting input to type torch.cuda
        images_batch.requires_grad_()
        images_batch = images_batch.to(device)

        # Converting targets to type torch.cuda
        target.requires_grad_()
        target = target.to(device)

        out = model(images_batch) # Forward pass

        loss += criterion(out, target) # Computing the loss and summing over it

        print(f'Batch : {i+1} Aggregate batch loss : {loss}')
        
        pred = out.data
        
        correct += pred.eq(target.data).sum()
        
    # Computing average loss    
    loss /= len(val_dataloader.dataset)
        
    print(f'\nAverage loss: {loss}, Accuracy: {correct}/{len(val_dataloader.dataset)},{100*(correct/len(val_dataloader.dataset))}')
    
    print(f'Epoch : {epoch+1} avg loss : {loss}')



In [0]:
validate(1) # As we dont need more than one epoch for validating